In [1]:
import numpy as np
import pandas as pd
import operator
import re
import os 

import praw
import emoji
from concurrent.futures import ThreadPoolExecutor, as_completed

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
COMMON_WORDS = set(stopwords.words("english"))

In [2]:
class Get_subreddit():
    def __init__(self, subreddit):
        reddit = praw.Reddit(
            client_id= os.environ['CLIENT_ID_REDDIT'], 
            client_secret= os.environ['CLIENT_SECRET_REDDIT'], 
            user_agent="hsabbar",
        )
        self.hot = reddit.subreddit(subreddit).hot(limit=500)
        self.max_comments = 100
        self.data = []
        self.columns = ['title','id','score', 'date', 'author', 'content', 'num_comments', 'comments'] 
        
    def get_data(self, submission):
        data_tmp = []
        comment_data = []
        data_tmp = [submission.title, submission.id, submission.score,submission.created_utc, submission.author, 
                     submission.selftext, len(submission.comments)]

        submission.comments.replace_more(limit=self.max_comments)
        comment_data = [top_level_comment.body for top_level_comment in submission.comments[:self.max_comments]]
        

        """
        for top_level_comment in submission.comments[:self.max_comments]: 
            comment_data.append(top_level_comment.body)
        if len(comment_data) < self.max_comments:
            comment_data = comment_data + [None] * (self.max_comments - len(comment_data))
        """
        return data_tmp + [' '.join(comment_data)]
    
    def get_data_multi_thread(self):
        processes = []
        ccccccc

        self.data = [task.result() for task in as_completed(processes)]
        
        #print(self.data)
        return pd.DataFrame(self.data, columns = self.columns)

In [80]:
%%time
reddit_data = Get_subreddit("Bitcoin")
df = reddit_data.get_data_multi_thread()
df.to_csv("subReddit-data-Bitcion.csv", sep=';', encoding='utf-8')
#df.head()

Wall time: 3.01 s


In [3]:
df2 = pd.read_csv("subReddit-data-Bitcion.csv", sep=';')
df2.head()

,Unnamed: 0,title,id,score,date,author,content,num_comments,comments
0,0,Best hardware wallet?,nla5z5,16,1.622011e+09,Negative_Comedian870,Hey guys. \n\nSeeing lots of people on the Led...,16,"I personaly use trezor, but i think the only p..."
1,1,"IT Giant Globant Joins Buys $500,000 Worth of ...",nkmhp2,36,1.621939e+09,VAMPXIII,NaN,10,People here acting like 10 BTC is a small quan...
2,2,Cryptocurrency Had a Dire Weekend. Why?,nl3nd2,1,1.621988e+09,SnooSketches3367,NaN,1,Novagrass ? talked about 5 factors on utube's ...
3,3,Wash-sale regulation doesn't apply to Bitcoin ...,nlra50,1,1.622063e+09,Wishy_washy_Though,"""Bitcoin crash opens door to a tax loophole fo...",1,"Bob buys 1 BTC at $60K. Later, Bob sells 1 BTC..."
4,4,Bitcoin Surges Over $40k After Bitcoin Mining ...,nlkf4q,0,1.622045e+09,crypto_pub,NaN,5,Yea no Bitcoin would test 40k no matter the la...


In [4]:
class Data_processing():
    def __init__(self, df):
        self.df = df
        self.columns_text = ['title', 'content', 'comments'] #+ [f'comment{i}' for i in range(1, 101)]
        self.del_url = re.compile(r'https(.*?)(\s|$)')
    
    
    def my_stopwords(self, text):
        text_tokens = word_tokenize(text)
        data = [word for word in text_tokens if word not in COMMON_WORDS and len(word) > 1]
        return " ".join(data)

    
        
    def nlp_pipeline(self, text):
        if text is not None :
            
            text = str(text).lower()
            #text = text.replace('\n', ' ').replace('\r', '')
            #text = ' '.join(text.split())
            #text = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", ext)
            #text = re.sub(r"(\s\-\s|-$)", "", text)
            #text = re.sub(r"\&\S*\s", " ", text)
            #text = re.sub(r"['&@¨=“”’‘+—^.;:-_~!-/\{\}*=\%?,#<>\"()\[\]]", "", text)
            
            #for url in self.del_url.findall(text):
             #   text = re.sub(f'https{url[0]}', "", text)
            text = emoji.get_emoji_regexp().sub("", text) 
            
            #text = self.my_stopwords(text)
            return text
        else :
            pass
    
    def clean_data(self):
        for col_txt in self.columns_text :
            #print(col_txt)
            self.df[col_txt] = self.df[col_txt].apply(lambda x:  self.nlp_pipeline(x))
        
        return self.df

In [5]:
%%time
data_proc = Data_processing(df2)
df2 = data_proc.clean_data()
df2.head()

Wall time: 7.68 s


,Unnamed: 0,title,id,score,date,author,content,num_comments,comments
0,0,best hardware wallet?,nla5z5,16,1.622011e+09,Negative_Comedian870,hey guys. \n\nseeing lots of people on the led...,16,"i personaly use trezor, but i think the only p..."
1,1,"it giant globant joins buys $500,000 worth of ...",nkmhp2,36,1.621939e+09,VAMPXIII,nan,10,people here acting like 10 btc is a small quan...
2,2,cryptocurrency had a dire weekend. why?,nl3nd2,1,1.621988e+09,SnooSketches3367,nan,1,novagrass ? talked about 5 factors on utube's ...
3,3,wash-sale regulation doesn't apply to bitcoin ...,nlra50,1,1.622063e+09,Wishy_washy_Though,"""bitcoin crash opens door to a tax loophole fo...",1,"bob buys 1 btc at $60k. later, bob sells 1 btc..."
4,4,bitcoin surges over $40k after bitcoin mining ...,nlkf4q,0,1.622045e+09,crypto_pub,nan,5,yea no bitcoin would test 40k no matter the la...


In [6]:
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment.util import *

from nltk import tokenize

In [7]:
class Label_nltk():
    def __init__(self, df):
        self.df = df
        self.colmuns_text = ["title", "content", "comments"]
        
    def label(self, colmun):
        labels = []
        for col in self.df.colmun:
            suntances = tokenize.sent_tokenize(col)
            #label_last = None
            for suntance in suntances:
                scor = sia.polarity_scores(suntance)
                label = max(scor.items(), key=operator.itemgetter(1))

                #if label == label_last :
                #    print(label, suntance)
                #label_last = label
            labels.append(label)
        self.df[f'label_{colmun}'] = labels
        
    def make_lable(self):
        with ThreadPoolExecutor(max_workers=62) as executor:
            for colmun in self.colmuns_text:
                executor.submit(self.label, colmun)

In [8]:
%%time
data_labels = Label_nltk(df2)
data_labels.make_lable()
df2.head()

Wall time: 8.14 ms


,Unnamed: 0,title,id,score,date,author,content,num_comments,comments
0,0,best hardware wallet?,nla5z5,16,1.622011e+09,Negative_Comedian870,hey guys. \n\nseeing lots of people on the led...,16,"i personaly use trezor, but i think the only p..."
1,1,"it giant globant joins buys $500,000 worth of ...",nkmhp2,36,1.621939e+09,VAMPXIII,nan,10,people here acting like 10 btc is a small quan...
2,2,cryptocurrency had a dire weekend. why?,nl3nd2,1,1.621988e+09,SnooSketches3367,nan,1,novagrass ? talked about 5 factors on utube's ...
3,3,wash-sale regulation doesn't apply to bitcoin ...,nlra50,1,1.622063e+09,Wishy_washy_Though,"""bitcoin crash opens door to a tax loophole fo...",1,"bob buys 1 btc at $60k. later, bob sells 1 btc..."
4,4,bitcoin surges over $40k after bitcoin mining ...,nlkf4q,0,1.622045e+09,crypto_pub,nan,5,yea no bitcoin would test 40k no matter the la...


In [30]:
%%time
def label(df):
    labels = []
    sia = SentimentIntensityAnalyzer()
    for col in df.comments: 
        paragraph = tokenize.sent_tokenize(str(col))
        label_last = ('x', 12)
        paragraph_labels = []
        print('-------------')
        for suntance in paragraph:
            scor = sia.polarity_scores(suntance)
            label = max(scor.items(), key=operator.itemgetter(1))
            
            if label[0] != label_last[0] and label not in paragraph_labels:
                paragraph_labels.append(label)
                
            label_last = label
            print(paragraph_labels)
        labels.append(paragraph_labels)
        
    df[f'label_comments'] = labels
    
    
label(df2)
df2.head()

-------------
[('neu', 0.818)]
[('neu', 0.818)]
[('neu', 0.818), ('neg', 0.61)]
[('neu', 0.818), ('neg', 0.61), ('neu', 0.877)]
[('neu', 0.818), ('neg', 0.61), ('neu', 0.877), ('compound', 0.8591)]
[('neu', 0.818), ('neg', 0.61), ('neu', 0.877), ('compound', 0.8591), ('neu', 1.0)]
[('neu', 0.818), ('neg', 0.61), ('neu', 0.877), ('compound', 0.8591), ('neu', 1.0)]
[('neu', 0.818), ('neg', 0.61), ('neu', 0.877), ('compound', 0.8591), ('neu', 1.0)]
[('neu', 0.818), ('neg', 0.61), ('neu', 0.877), ('compound', 0.8591), ('neu', 1.0)]
[('neu', 0.818), ('neg', 0.61), ('neu', 0.877), ('compound', 0.8591), ('neu', 1.0)]
[('neu', 0.818), ('neg', 0.61), ('neu', 0.877), ('compound', 0.8591), ('neu', 1.0)]
[('neu', 0.818), ('neg', 0.61), ('neu', 0.877), ('compound', 0.8591), ('neu', 1.0)]
[('neu', 0.818), ('neg', 0.61), ('neu', 0.877), ('compound', 0.8591), ('neu', 1.0)]
[('neu', 0.818), ('neg', 0.61), ('neu', 0.877), ('compound', 0.8591), ('neu', 1.0)]
[('neu', 0.818), ('neg', 0.61), ('neu', 0.877)

[('neu', 0.847)]
[('neu', 0.847)]
[('neu', 0.847)]
[('neu', 0.847)]
[('neu', 0.847)]
[('neu', 0.847), ('neg', 0.612)]
[('neu', 0.847), ('neg', 0.612), ('neu', 0.741)]
[('neu', 0.847), ('neg', 0.612), ('neu', 0.741)]
[('neu', 0.847), ('neg', 0.612), ('neu', 0.741)]
[('neu', 0.847), ('neg', 0.612), ('neu', 0.741)]
-------------
[('neu', 1.0)]
-------------
[('neu', 1.0)]
[('neu', 1.0)]
[('neu', 1.0), ('compound', 0.8313)]
[('neu', 1.0), ('compound', 0.8313)]
[('neu', 1.0), ('compound', 0.8313)]
[('neu', 1.0), ('compound', 0.8313)]
[('neu', 1.0), ('compound', 0.8313), ('compound', 0.8591)]
[('neu', 1.0), ('compound', 0.8313), ('compound', 0.8591), ('neu', 0.847)]
[('neu', 1.0), ('compound', 0.8313), ('compound', 0.8591), ('neu', 0.847), ('compound', 0.7227)]
[('neu', 1.0), ('compound', 0.8313), ('compound', 0.8591), ('neu', 0.847), ('compound', 0.7227), ('neu', 0.58)]
[('neu', 1.0), ('compound', 0.8313), ('compound', 0.8591), ('neu', 0.847), ('compound', 0.7227), ('neu', 0.58)]
[('neu', 1

[('neu', 0.704)]
[('neu', 0.704)]
[('neu', 0.704)]
[('neu', 0.704)]
[('neu', 0.704)]
[('neu', 0.704), ('compound', 0.743)]
[('neu', 0.704), ('compound', 0.743), ('neu', 1.0)]
[('neu', 0.704), ('compound', 0.743), ('neu', 1.0)]
[('neu', 0.704), ('compound', 0.743), ('neu', 1.0), ('pos', 1.0)]
[('neu', 0.704), ('compound', 0.743), ('neu', 1.0), ('pos', 1.0), ('neu', 0.617)]
-------------
[('neu', 1.0)]
-------------
[('neu', 1.0)]
[('neu', 1.0)]
[('neu', 1.0)]
[('neu', 1.0)]
[('neu', 1.0)]
[('neu', 1.0)]
[('neu', 1.0)]
-------------
[('neu', 0.814)]
[('neu', 0.814)]
[('neu', 0.814)]
[('neu', 0.814)]
[('neu', 0.814)]
[('neu', 0.814)]
[('neu', 0.814)]
[('neu', 0.814)]
[('neu', 0.814)]
[('neu', 0.814)]
[('neu', 0.814)]
[('neu', 0.814)]
-------------
[('neu', 0.824)]
[('neu', 0.824)]
[('neu', 0.824)]
[('neu', 0.824)]
[('neu', 0.824), ('compound', 0.8979)]
-------------
[('neu', 0.893)]
[('neu', 0.893)]
[('neu', 0.893)]
[('neu', 0.893)]
-------------
[('neu', 0.588)]
[('neu', 0.588)]
[('neu',

[('compound', 0.7269), ('neu', 1.0), ('compound', 0.7992), ('compound', 0.7845), ('neu', 0.777), ('compound', 0.5719), ('neu', 0.828), ('compound', 0.5994), ('neu', 0.783), ('pos', 0.556), ('neu', 0.937), ('compound', 0.8355), ('compound', 0.9501), ('neg', 0.891), ('neu', 0.765), ('pos', 1.0), ('neu', 0.632), ('neg', 0.598), ('neg', 0.518), ('neu', 0.724), ('neg', 0.533), ('neu', 0.617), ('pos', 0.762), ('compound', 0.8955), ('neu', 0.626), ('neg', 0.667), ('neu', 0.641), ('compound', 0.8478), ('neu', 0.616), ('compound', 0.7503), ('neg', 0.636), ('neu', 0.687), ('compound', 0.6908)]
[('compound', 0.7269), ('neu', 1.0), ('compound', 0.7992), ('compound', 0.7845), ('neu', 0.777), ('compound', 0.5719), ('neu', 0.828), ('compound', 0.5994), ('neu', 0.783), ('pos', 0.556), ('neu', 0.937), ('compound', 0.8355), ('compound', 0.9501), ('neg', 0.891), ('neu', 0.765), ('pos', 1.0), ('neu', 0.632), ('neg', 0.598), ('neg', 0.518), ('neu', 0.724), ('neg', 0.533), ('neu', 0.617), ('pos', 0.762), ('

[('neu', 0.811)]
[('neu', 0.811)]
[('neu', 0.811), ('neg', 0.778)]
[('neu', 0.811), ('neg', 0.778), ('compound', 0.765)]
[('neu', 0.811), ('neg', 0.778), ('compound', 0.765), ('neu', 1.0)]
[('neu', 0.811), ('neg', 0.778), ('compound', 0.765), ('neu', 1.0)]
-------------
[('neu', 0.726)]
[('neu', 0.726)]
[('neu', 0.726)]
[('neu', 0.726)]
[('neu', 0.726)]
[('neu', 0.726)]
[('neu', 0.726), ('neg', 0.0)]
-------------
[('neu', 1.0)]
-------------
[('neu', 0.485)]
[('neu', 0.485)]
[('neu', 0.485)]
[('neu', 0.485)]
[('neu', 0.485)]
[('neu', 0.485)]
[('neu', 0.485)]
[('neu', 0.485)]
[('neu', 0.485), ('compound', 0.7666)]
[('neu', 0.485), ('compound', 0.7666), ('neu', 0.656)]
[('neu', 0.485), ('compound', 0.7666), ('neu', 0.656), ('pos', 0.643)]
[('neu', 0.485), ('compound', 0.7666), ('neu', 0.656), ('pos', 0.643), ('neu', 0.778)]
[('neu', 0.485), ('compound', 0.7666), ('neu', 0.656), ('pos', 0.643), ('neu', 0.778)]
[('neu', 0.485), ('compound', 0.7666), ('neu', 0.656), ('pos', 0.643), ('neu',

...

[('neu', 0.93)]
-------------
[('neu', 0.671)]
[('neu', 0.671)]
[('neu', 0.671)]
[('neu', 0.671)]
[('neu', 0.671)]
[('neu', 0.671)]
[('neu', 0.671)]
[('neu', 0.671)]
[('neu', 0.671)]
[('neu', 0.671)]
[('neu', 0.671)]
[('neu', 0.671)]
[('neu', 0.671)]
[('neu', 0.671)]
[('neu', 0.671), ('compound', 0.743)]
[('neu', 0.671), ('compound', 0.743), ('neu', 1.0)]
[('neu', 0.671), ('compound', 0.743), ('neu', 1.0)]
[('neu', 0.671), ('compound', 0.743), ('neu', 1.0), ('compound', 0.8307)]
[('neu', 0.671), ('compound', 0.743), ('neu', 1.0), ('compound', 0.8307)]
[('neu', 0.671), ('compound', 0.743), ('neu', 1.0), ('compound', 0.8307)]
[('neu', 0.671), ('compound', 0.743), ('neu', 1.0), ('compound', 0.8307)]
[('neu', 0.671), ('compound', 0.743), ('neu', 1.0), ('compound', 0.8307)]
[('neu', 0.671), ('compound', 0.743), ('neu', 1.0), ('compound', 0.8307)]
[('neu', 0.671), ('compound', 0.743), ('neu', 1.0), ('compound', 0.8307)]
[('neu', 0.671), ('compound', 0.743), ('neu', 1.0), ('compound', 0.8307)

[('neu', 1.0), ('compound', 0.8718), ('compound', 0.8765), ('neu', 0.774), ('pos', 1.0), ('pos', 0.476), ('neg', 0.589), ('neu', 0.656), ('compound', 0.7864), ('neu', 0.621), ('compound', 0.6249), ('neg', 0.796)]
[('neu', 1.0), ('compound', 0.8718), ('compound', 0.8765), ('neu', 0.774), ('pos', 1.0), ('pos', 0.476), ('neg', 0.589), ('neu', 0.656), ('compound', 0.7864), ('neu', 0.621), ('compound', 0.6249), ('neg', 0.796), ('neu', 0.383)]
[('neu', 1.0), ('compound', 0.8718), ('compound', 0.8765), ('neu', 0.774), ('pos', 1.0), ('pos', 0.476), ('neg', 0.589), ('neu', 0.656), ('compound', 0.7864), ('neu', 0.621), ('compound', 0.6249), ('neg', 0.796), ('neu', 0.383), ('compound', 0.6369)]
[('neu', 1.0), ('compound', 0.8718), ('compound', 0.8765), ('neu', 0.774), ('pos', 1.0), ('pos', 0.476), ('neg', 0.589), ('neu', 0.656), ('compound', 0.7864), ('neu', 0.621), ('compound', 0.6249), ('neg', 0.796), ('neu', 0.383), ('compound', 0.6369), ('neu', 0.546)]
[('neu', 1.0), ('compound', 0.8718), ('c

[('neu', 1.0), ('neg', 0.423), ('neu', 0.79), ('compound', 0.6486)]
[('neu', 1.0), ('neg', 0.423), ('neu', 0.79), ('compound', 0.6486)]
[('neu', 1.0), ('neg', 0.423), ('neu', 0.79), ('compound', 0.6486)]
[('neu', 1.0), ('neg', 0.423), ('neu', 0.79), ('compound', 0.6486)]
[('neu', 1.0), ('neg', 0.423), ('neu', 0.79), ('compound', 0.6486)]
[('neu', 1.0), ('neg', 0.423), ('neu', 0.79), ('compound', 0.6486)]
[('neu', 1.0), ('neg', 0.423), ('neu', 0.79), ('compound', 0.6486)]
[('neu', 1.0), ('neg', 0.423), ('neu', 0.79), ('compound', 0.6486)]
[('neu', 1.0), ('neg', 0.423), ('neu', 0.79), ('compound', 0.6486)]
[('neu', 1.0), ('neg', 0.423), ('neu', 0.79), ('compound', 0.6486), ('compound', 0.6249)]
[('neu', 1.0), ('neg', 0.423), ('neu', 0.79), ('compound', 0.6486), ('compound', 0.6249), ('neu', 0.828)]
[('neu', 1.0), ('neg', 0.423), ('neu', 0.79), ('compound', 0.6486), ('compound', 0.6249), ('neu', 0.828)]
[('neu', 1.0), ('neg', 0.423), ('neu', 0.79), ('compound', 0.6486), ('compound', 0.624

[('neu', 1.0), ('compound', 0.8591), ('neu', 0.929), ('compound', 0.6369)]
[('neu', 1.0), ('compound', 0.8591), ('neu', 0.929), ('compound', 0.6369)]
[('neu', 1.0), ('compound', 0.8591), ('neu', 0.929), ('compound', 0.6369)]
[('neu', 1.0), ('compound', 0.8591), ('neu', 0.929), ('compound', 0.6369)]
[('neu', 1.0), ('compound', 0.8591), ('neu', 0.929), ('compound', 0.6369)]
[('neu', 1.0), ('compound', 0.8591), ('neu', 0.929), ('compound', 0.6369)]
[('neu', 1.0), ('compound', 0.8591), ('neu', 0.929), ('compound', 0.6369)]
[('neu', 1.0), ('compound', 0.8591), ('neu', 0.929), ('compound', 0.6369)]
[('neu', 1.0), ('compound', 0.8591), ('neu', 0.929), ('compound', 0.6369)]
[('neu', 1.0), ('compound', 0.8591), ('neu', 0.929), ('compound', 0.6369)]
[('neu', 1.0), ('compound', 0.8591), ('neu', 0.929), ('compound', 0.6369)]
[('neu', 1.0), ('compound', 0.8591), ('neu', 0.929), ('compound', 0.6369)]
[('neu', 1.0), ('compound', 0.8591), ('neu', 0.929), ('compound', 0.6369)]
[('neu', 1.0), ('compound

,Unnamed: 0,title,id,score,date,author,content,num_comments,comments,label_comments
0,0,best hardware wallet?,nla5z5,16,1.622011e+09,Negative_Comedian870,hey guys. \n\nseeing lots of people on the led...,16,"i personaly use trezor, but i think the only p...","[(neu, 0.818), (neg, 0.61), (neu, 0.877), (com..."
1,1,"it giant globant joins buys $500,000 worth of ...",nkmhp2,36,1.621939e+09,VAMPXIII,nan,10,people here acting like 10 btc is a small quan...,"[(neu, 0.794), (compound, 0.7579), (neu, 1.0)]"
2,2,cryptocurrency had a dire weekend. why?,nl3nd2,1,1.621988e+09,SnooSketches3367,nan,1,novagrass ? talked about 5 factors on utube's ...,"[(neu, 1.0)]"
3,3,wash-sale regulation doesn't apply to bitcoin ...,nlra50,1,1.622063e+09,Wishy_washy_Though,"""bitcoin crash opens door to a tax loophole fo...",1,"bob buys 1 btc at $60k. later, bob sells 1 btc...","[(neu, 1.0), (compound, 0.802)]"
4,4,bitcoin surges over $40k after bitcoin mining ...,nlkf4q,0,1.622045e+09,crypto_pub,nan,5,yea no bitcoin would test 40k no matter the la...,"[(neu, 0.513), (compound, 0.8207), (neu, 0.874..."
